In [1]:
# Import những thư viện cần thiết

import numpy as np
import re
import nltk
from sklearn.datasets import load_files
import pickle
from pyvi import ViTokenizer

In [12]:
# Load training data file

training_data = load_files(r"./data1", encoding="utf-8")
X, Y = training_data.data, training_data.target

In [13]:
# Load file chứa các từ stop_words

stop_ws = []
with open("vnstopword.txt", encoding="utf-8") as f :
    text = f.read()
    for word in text.split('\n') :  #Tách ra mỗi dòng là 1 từ stopword riêng lẻ
        stop_ws.append(word)
    f.close()

In [14]:
# Tiền xử lý cho tập dữ liệu training
documents = []
sum_word = 0
for sen in range(0, len(X)):
    
    # remove stop word
    document = []
    for word in str(X[sen]).split() :
        sum_word = sum_word + 1         # Đếm tổng số từ trong tập dữ liệu
        if (word not in stop_ws) :
            if ("_" in word) or (word.isalpha() == True):
                document.append(word)
    #print(document)
    document = ' '.join(document)

    replace_list = {
        'òa': 'oà', 'óa': 'oá', 'ỏa': 'oả', 'õa': 'oã', 'ọa': 'oạ', 'òe': 'oè', 'óe': 'oé','ỏe': 'oẻ',
        'õe': 'oẽ', 'ọe': 'oẹ', 'ùy': 'uỳ', 'úy': 'uý', 'ủy': 'uỷ', 'ũy': 'uỹ','ụy': 'uỵ', 'uả': 'ủa',
        'ả': 'ả', 'ố': 'ố', 'u´': 'ố','ỗ': 'ỗ', 'ồ': 'ồ', 'ổ': 'ổ', 'ấ': 'ấ', 'ẫ': 'ẫ', 'ẩ': 'ẩ',
        'ầ': 'ầ', 'ỏ': 'ỏ', 'ề': 'ề','ễ': 'ễ', 'ắ': 'ắ', 'ủ': 'ủ', 'ế': 'ế', 'ở': 'ở', 'ỉ': 'ỉ',
        'ẻ': 'ẻ', 'àk': u' à ','aˋ': 'à', 'iˋ': 'ì', 'ă´': 'ắ','ử': 'ử', 'e˜': 'ẽ', 'y˜': 'ỹ', 'a´': 'á',
        ' okey ': ' ok ', 'ôkê': ' ok ', 'oki': ' ok ', ' oke ':  ' ok ',' okay':' ok ','okê':' ok ',
        ' tks ': u' cám ơn ', 'thks': u' cám ơn ', 'thanks': u' cám ơn ', 'ths': u' cám ơn ', 'thank': u' cám ơn ',
        ' kg ': u' không ','not': u' không ', u' kg ': u' không ', ' k ': u' không ',' kh ':u' không ',' kô ':u' không ',' hok ':u' không ',' kp ': u' không phải ',u' kô ': u' không ', ' ko ': u' không ', u' ko ': u' không ', u' k ': u' không ', 'khong': u' không ', u' hok ': u' không ',' hong ':u' không',
        'vs': u' với ', 'wa': ' quá ', 'wá': u' quá', 'j': u' gì ', '“': ' ',' đx ': u' được ', 'dk': u' được ', 'dc': u' được ', 'đk': u' được ',
        'đc': u' được ',' thick ': u' thích ', 'store': u' cửa hàng ',
        'shop': u' cửa hàng ', 'sp': u' sản phẩm ', 'gud': u' tốt ','god': u' tốt ','wel done':' tốt ', 'good': u' tốt ', 'gút': u' tốt ',
        ' sấu ': u' xấu ','gut': u' tốt ', u' tot ': u' tốt ', u' nice ': u' tốt ', 'perfect': 'rất tốt', 'bt': u' bình thường ',
        'time': u' thời gian ', 'qá': u' quá ', u' ship ': u' giao hàng ', u' m ': u' mình ', u' mik ': u' mình ',
        'date': u' hạn sử dụng ', 'hsd': u' hạn sử dụng ','quickly': u' nhanh ', 'quick': u' nhanh ','fast': u' nhanh ','delivery': u' giao hàng ',u' síp ': u' giao hàng ',
        'beautiful': u' đẹp tuyệt vời ', u' tl ': u' trả lời ', u' r ': u' rồi ', u' shopE ': u' cửa hàng ',u' order ': u' đặt hàng ',
        'chất lg': u' chất lượng ',u' sd ': u' sử dụng ',u' dt ': u' điện thoại ',u' nt ': u' nhắn tin ',u' tl ': u' trả lời ',u' sài ': u' xài ',u'bjo':u' bao giờ ',
        'thik': u' thích ',u' sop ': u' cửa hàng ', ' fb ': ' facebook ', ' face ': ' facebook ', ' very ': u' rất ',u'quả ng ':u' quảng  ',
        ' dep ': u' đẹp ',u' xau ': u' xấu ','delicious': u' ngon ', u'hàg': u' hàng ', u'qủa': u' quả ','iu': u' yêu ','pv':u' phục vụ','nv':u' nhân viên','nc':u' nước',
        u' hó ':u' khó',u' hắc ':u' khắc ','_': ' ', u' ng ':u' người '}

    for k, v in replace_list.items():
        document = document.replace(k, v)
    
    
    
    # Loại bỏ Hashtag
    document = re.sub(r'#([^\s]+)', '', document)
    # Loại bỏ các URL
    document = re.sub(r'http([^\s]+)', '', document)
    # Loại bỏ number có trong comment
    document = re.sub(r'\d+', '', document)
    #Remove duplicated characters: vd: đẹppppppp
    document = re.sub(r'([A-Z])\1+', lambda m: m.group(1).upper(), document, flags=re.IGNORECASE)
    # Loại bỏ các dấu câu, các ký tự đặc biệt (Emoji, icon cũng bị remove)
    document = re.sub(r'[^\w\s]','', document)
    # Chuyển chữ in hoa thành chữ thường
    document = document.lower()
    # Loại bỏ các ký tự dư thừa đơn lẻ (Nhiều ký tự đơn lẻ liền nhau kg xử lý được)
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    # Loại bỏ các Emoji, icon
    document = re.sub(r'[^\w\s,]', '', document)
    # Loại bỏ nhiều khoảng trắng dư thừa
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Tokenization
    document = ViTokenizer.tokenize(document)
    
    document = ''.join(document)
    
    documents.append(document)

In [21]:
#print(documents)

In [15]:
# Số lượng word trong tập dữ liệu
print(sum_word)

2962235


In [16]:
# Phân chia dữ liệu thành 2 phần: Training và Testing
# X là data, Y là label

from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stop_ws)
X = tfidfconverter.fit_transform(documents).toarray()

from sklearn.model_selection import train_test_split
X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [17]:
# Xây dựng mô hình huấn luyện bằng NaiveBayes

import time
from sklearn.naive_bayes import BernoulliNB

# Thêm biến này vào để kiểm tra thời gian huấn luyện
start_NB = time.time()


classifier_NB = BernoulliNB(binarize=None)
classifier_NB.fit(X_Train, Y_Train)

end_NB = time.time()
time_NB = end_NB-start_NB

#Show ra thời gian huấn luyện của mô hình
#print("Thời gian huấn luyện: %s second" % str(end-start))
print("Thời gian huấn luyện: ", str(time_NB), " giây")

Thời gian huấn luyện:  0.33776378631591797  giây


In [18]:
# Xây dựng mô hình huấn luyện bằng Random Forest
import time
from sklearn.ensemble import RandomForestClassifier

# Thêm biến này vào để kiểm tra thời gian huấn luyện
start_RF = time.time()

classifier_RF = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier_RF.fit(X_Train, Y_Train) 

end_RF = time.time()
time_RF = end_RF-start_RF

#Show ra thời gian huấn luyện của mô hình
#print("Thời gian huấn luyện: %s second" % str(end-start))
print("Thời gian huấn luyện: ", str(time_RF), " giây")

Thời gian huấn luyện:  502.6755259037018  giây


In [19]:
# Xây dựng mô hình huấn luyện bằng SVM
import time

from sklearn.svm import SVC

# Thêm biến này vào để kiểm tra thời gian huấn luyện
start_SVM = time.time()

classifier_SVM = SVC(kernel = 'linear', random_state = 0)
classifier_SVM.fit(X_Train, Y_Train)


end_SVM = time.time()
time_SVM = end_SVM-start_SVM

#Show ra thời gian huấn luyện của mô hình
#print("Thời gian huấn luyện: %s second" % str(end-start))
print("Thời gian huấn luyện: ", str(time_SVM), " giây")

Thời gian huấn luyện:  516.385575056076  giây


In [27]:
# Xây dựng mô hình huấn luyện bằng Logistic Regression

from sklearn.linear_model import LogisticRegression
start = time.time()

classifier_LR = LogisticRegression(fit_intercept=True, n_jobs=4)
classifier_LR.fit(X_Train, Y_Train)

end = time.time()

print("Thời gian huấn luyện Logistic Regression: ", str(end-start), " giây")

Thời gian huấn luyện Logistic Regression:  20.218449592590332  giây


In [28]:
# Xây dựng mô hình huấn luyện bằng KNN

from sklearn.neighbors import KNeighborsClassifier

start = time.time()

classifier_KNN = KNeighborsClassifier(n_neighbors=5)
classifier_KNN.fit(X_Train, Y_Train)

end = time.time()

print("Thời gian huấn luyện KNN: ", str(end-start), " giây")

Thời gian huấn luyện KNN:  5.6025390625  giây


In [20]:
# Predicting Sentiment

y_pred_NB = classifier_NB.predict(X_Test)

y_pred_RF = classifier_RF.predict(X_Test)

y_pred_SVM = classifier_SVM.predict(X_Test)

#y_Pred_LR = classifier_LR.predict(X_Test)

#y_Pred_KNN = classifier_KNN.predict(X_Test)


In [21]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

#print('Accuracy LogisticRegression: ', accuracy_score(Y_Test, y_Pred_LR))
print('Accuracy of Naive Bayes:',accuracy_score(Y_Test, y_pred_NB))
print('Accuracy of Random Forest:',accuracy_score(Y_Test, y_pred_RF))
print('Accuracy of SVM:',accuracy_score(Y_Test, y_pred_SVM))
#print('Accuracy KNN: ', accuracy_score(Y_Test, y_Pred_KNN))

Accuracy of Naive Bayes: 0.8336666666666667
Accuracy of Random Forest: 0.8488888888888889
Accuracy of SVM: 0.868


In [31]:
#print('Training time of NB:',time_NB)
#print('Training time of RF:',time_RF)
#print('Training time of SVM:',time_SVM)